In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
import cv2 as cv

Using TensorFlow backend.


In [2]:
number_of_fonts = 5
number_characters = 62 #26 lowercase, 26 uppercase, 10 digits 
number_special_characters = 3 #period, semicollon ...


total_characters = number_characters + number_special_characters

x_train = []
y_train = []

In [3]:
def char_to_onehot(character):
    legit_char=True
    one_hot = np.zeros(total_characters)
    identifiers = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 
               'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26,
               'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 
               'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52,
               '0': 53, '1': 54, '2': 55, '3': 56, '4': 57,'5': 58, '6': 59, '7': 60, '8': 61, '9': 62, 'prd': 63, 'cln': 64, 'smcln': 65
              }    
    
    try:
        index = identifiers[character] - 1
        one_hot[index] = 1
    except:
        legit_char = False
    
    return legit_char, one_hot


In [4]:
def shuffle_in_unison(a, b): #Credits to someone on StackOverflow
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [5]:
import os
import cv2 as cv
import random

file_path = 'image_dataset/'
x_data_img = []
y_data_img = []
IMG_ROWS = 45
IMG_COLS = 45

file_names = []
for file_name in sorted(os.listdir(file_path)):
    file_names.append(file_name)

number_of_files = len(file_names)
random.shuffle(file_names)

for i in range(number_of_files):
    file_name = file_names[i]
    img = cv.imread(file_path+file_name)
    
    img = 255 - img
    img = cv.resize(img, dsize=(IMG_ROWS, IMG_COLS), interpolation=cv.INTER_CUBIC)
            
    splits = file_name.split('%') #gives something like - ['char_1', 'A.tiff']
    splits = splits[1].split('.') #A.tiff is split into ['A', 'tiff']
    character_label = splits[0]   #Taking A from index O of above splits
    
    legit_char, one_hot = char_to_onehot(character_label)
    if(legit_char):
        x_data_img.append(img)
        y_data_img.append(one_hot)
    else:
        print('Problem occured in file name labelling when accessing file '+str(file_name)+'\n Follow convention - ***%A.tiff or ***%A.png')

x_data_img = np.asarray(x_data_img)
y_data_img = np.asarray(y_data_img)

shuffle_in_unison(x_data_img, y_data_img)

print('Number of images and its dimensions: '+str(x_data_img.shape))
print('Number of one hot vectors:           '+str(y_data_img.shape))

Number of images and its dimensions: (1625, 45, 45, 3)
Number of one hot vectors:           (1625, 65)


In [6]:
x_train = x_data_img[ :1200]
y_train = y_data_img[ :1200]

x_test = x_data_img[:40]
y_test = y_data_img[:40]

model = Sequential()
# input: 30x30 images with 1 channel -> (30, 30, 1) tensors.
# this applies 32 convolution filters of size 3x3 each.

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(45, 45, 3)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(65, activation='softmax'))

sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x_train, y_train, batch_size=32, epochs=300)
score = model.evaluate(x_test, y_test, batch_size=32)
print(score)

Epoch 1/300
1200/1200 [==============================] - 18s 15ms/step - loss: 15.3932
Epoch 2/300
1200/1200 [==============================] - 0s 405us/step - loss: 12.9175
Epoch 3/300
1200/1200 [==============================] - 0s 414us/step - loss: 4.6754
Epoch 4/300
1200/1200 [==============================] - 0s 406us/step - loss: 4.2516
Epoch 5/300
1200/1200 [==============================] - 0s 404us/step - loss: 4.1699
Epoch 6/300
1200/1200 [==============================] - 1s 435us/step - loss: 4.1502
Epoch 7/300
1200/1200 [==============================] - 0s 409us/step - loss: 4.1247
Epoch 8/300
1200/1200 [==============================] - 0s 405us/step - loss: 4.1208
Epoch 9/300
1200/1200 [==============================] - 0s 409us/step - loss: 4.0965
Epoch 10/300
1200/1200 [==============================] - 0s 405us/step - loss: 4.0753
Epoch 11/300
1200/1200 [==============================] - 0s 409us/step - loss: 4.0478
Epoch 12/300
1200/1200 [==========================

1200/1200 [==============================] - 1s 434us/step - loss: 0.8550
Epoch 95/300
1200/1200 [==============================] - 1s 452us/step - loss: 0.8355
Epoch 96/300
1200/1200 [==============================] - 0s 406us/step - loss: 0.8336
Epoch 97/300
1200/1200 [==============================] - 1s 427us/step - loss: 0.8305
Epoch 98/300
1200/1200 [==============================] - 1s 417us/step - loss: 0.8376
Epoch 99/300
1200/1200 [==============================] - 0s 404us/step - loss: 0.7593
Epoch 100/300
1200/1200 [==============================] - 0s 412us/step - loss: 0.8014
Epoch 101/300
1200/1200 [==============================] - 0s 405us/step - loss: 0.7718
Epoch 102/300
1200/1200 [==============================] - 1s 436us/step - loss: 0.7453
Epoch 103/300
1200/1200 [==============================] - 1s 456us/step - loss: 0.7939
Epoch 104/300
1200/1200 [==============================] - 1s 448us/step - loss: 0.7782
Epoch 105/300
1200/1200 [==========================

1200/1200 [==============================] - 0s 400us/step - loss: 0.3226
Epoch 188/300
1200/1200 [==============================] - 1s 433us/step - loss: 0.3240
Epoch 189/300
1200/1200 [==============================] - 0s 409us/step - loss: 0.3444
Epoch 190/300
1200/1200 [==============================] - 1s 427us/step - loss: 0.2989
Epoch 191/300
1200/1200 [==============================] - 0s 407us/step - loss: 0.3118
Epoch 192/300
1200/1200 [==============================] - 0s 403us/step - loss: 0.3397
Epoch 193/300
1200/1200 [==============================] - 0s 403us/step - loss: 0.3164
Epoch 194/300
1200/1200 [==============================] - 0s 410us/step - loss: 0.2919
Epoch 195/300
1200/1200 [==============================] - 0s 401us/step - loss: 0.3313
Epoch 196/300
1200/1200 [==============================] - 0s 404us/step - loss: 0.3437
Epoch 197/300
1200/1200 [==============================] - 0s 412us/step - loss: 0.3137
Epoch 198/300
1200/1200 [=====================

1200/1200 [==============================] - 1s 431us/step - loss: 0.1697
Epoch 280/300
1200/1200 [==============================] - 0s 415us/step - loss: 0.1748
Epoch 281/300
1200/1200 [==============================] - 1s 423us/step - loss: 0.2204
Epoch 282/300
1200/1200 [==============================] - 1s 419us/step - loss: 0.1851
Epoch 283/300
1200/1200 [==============================] - 1s 425us/step - loss: 0.2066
Epoch 284/300
1200/1200 [==============================] - 0s 405us/step - loss: 0.2011
Epoch 285/300
1200/1200 [==============================] - 1s 417us/step - loss: 0.1960
Epoch 286/300
1200/1200 [==============================] - 1s 426us/step - loss: 0.2125
Epoch 287/300
1200/1200 [==============================] - 0s 408us/step - loss: 0.2068
Epoch 288/300
1200/1200 [==============================] - 1s 450us/step - loss: 0.2101
Epoch 289/300
1200/1200 [==============================] - 1s 431us/step - loss: 0.2030
Epoch 290/300
1200/1200 [=====================

In [7]:
import pickle
filename = 'models/trained_model.sav'

#save the model
pickle.dump(model, open(filename, 'wb'))

#load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

In [9]:
x_test = x_data_img[:]
y_test = y_data_img[:]
number_samples = x_test.shape[0]
correct = 0


predictions = model.predict(x_test)
predictions = model.predict(x_test)

for i in range(number_samples):
    if(np.argmax(predictions[i]) == np.argmax(y_test[i])):
        correct+=1
print('Accuracy: '+str(100*correct/number_samples)+'%')

Accuracy: 95.56923076923077%
